In [6]:
import numpy as np
import pandas as pd

#Webscraping libraries
import requests
# from splinter import Browser
from bs4 import BeautifulSoup

#For text manipulation
import unicodedata

#Pickle!
import pickle

#Suppress warnings from showing
import warnings
warnings.filterwarnings('ignore')

In [7]:
def create_sql_table(dataframe, cursor, table_name, unique=True, verbose=True):

    '''
    Takes a dataframe and sqlite cursor and creates
    a table version of that dataframe
    in the sql database that the cursor is on
    '''
    #Get the column names (inc index)
    index_name = [dataframe.index.name]
    index_type = [dataframe.index.dtype]

    #If there's no specific name for the index, then give it a generic one
    if index_name == [None]:
        index_name = ['TableIndex']
        #Also assume that it's an integer primary key
        index_type = ['TEXT']

    columns = list(dataframe.columns)
    column_names = ", ".join(columns)
    columns = [f'{table_name}ID'] + index_name + columns

    #Get the data types of each
    column_types = [dataframe[i].dtype for i in dataframe.columns]

    #Transform the column types into SQL form
    type_mapper = {np.dtype('int64'): 'INTEGER',
                   np.dtype('float64'): 'REAL',
                   np.dtype('O'): 'TEXT'}

    column_types = [type_mapper[i] for i in column_types]
    column_types = ['INTEGER PRIMARY KEY'] + index_type + column_types

    #Zip the names and types together
    column_zip = list(zip(columns, column_types))

    cols_with_types = ""
    for i in column_zip:
        new_words = f'{i[0]} {i[1]}, '
        cols_with_types += new_words
    cols_with_types = cols_with_types[:-2]

    #If we have specified unique constraint, reflect this in the command
    if unique==True:
        sql_command = f"CREATE TABLE {table_name} ({cols_with_types}, unique ({column_names}));"
    else:
        sql_command = f"CREATE TABLE {table_name} ({cols_with_types});"

    if verbose==True:
        print(sql_command)

    #Include a try/except in case this is running for a second time...
    try:
        cursor.execute(sql_command)
        print('\nNew table added')
    except:
        print('\nTable not added - possibly already exists')

In [8]:
def populate_sql_from_dataframe(dataframe, sql_table, cursor, verbose=False):

    '''
    Takes a dataframe and the name of an sql table, and adds the dataframe
    rows to the bottom of the sql table associated with the cursor

    '''

    #Get the name of the index
    index_name = [dataframe.index.name]
    #If there's no specific name for the index, then give it a generic one
    if index_name == [None]:
        index_name = ['TableIndex']

    #Isolate dataframe columns and create a single string containing them
    columns = index_name + list(dataframe.columns)
    column_string = ', '.join(columns)

    #Iterate through the rows in the dataframe
    for i in range(len(dataframe)):
        try:
            index = dataframe.index[i]
            row = dataframe.iloc[i]
            #Isolate the values in each row
            values = [index] + [dataframe.iloc[i][col] for col in dataframe.columns]
            #Put inverted commas around text objects
            values_strings = [f'"{i}"' if type(i)==str else f'{i}' for i in values]
            #Combine all values into single string
            values_string = ", ".join(values_strings)

            #Create the command...
            command = f'''INSERT INTO {sql_table} ({column_string})
                       VALUES ({values_string});
                       '''
            if verbose==True:
                print(command)
            #And excecute it...
            cursor.execute(command)
        except:
            print(f'Error on row {i} - you may be trying to upload a duplicate row')

In [9]:

def sql(query, cursor):
    '''
    Takes an SQL query string, and outputs a
    dataframe representation of the query result.
    '''
    #Execute the sql query
    cursor.execute(query)

    #Get the query into a dataframe and set columns
    df_temp = pd.DataFrame(cursor.fetchall())
    df_temp.columns = [x[0] for x in cursor.description]

    #Set the sql id as the dataframe index
    index_column = df_temp.columns[0]
    df_temp.set_index(index_column, drop=True, inplace=True)

    return df_temp

In [10]:
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
results = requests.get(url).json()

#We create a blank dataframe to store team information
df_teams = pd.DataFrame(columns = ['TeamID','Team','ShortName','Strength'])

#We loop through all the team data taking the required information
for i in results['teams']:
    ID = i['id']
    team = i['name']
    short_name = i['short_name']
    strength = i['strength']

    df_temp = pd.DataFrame({'TeamID':[ID],
                            'Team':[team],
                            'ShortName':[short_name],
                            'Strength':[strength]})

    df_teams = pd.concat([df_teams, df_temp])

#We set the TeamID as the index for the dataframe
df_teams.set_index('TeamID',drop=True,inplace=True)


#Bring in the 'elements' dictionary from the json file
#This is part containing player informaion
players_json = results['elements']

#We create an empty dataframe with the columns we want
df_players = pd.DataFrame(columns = ['PlayerCode','PlayerID','FirstName',
                                     'WebName','Team','Position'])

#We should create a function that will remove accents from player names
#e.g. "à" should become "a", and so forth. This will help with commentary
#recognition later...
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

#We iterate through each of the players in the json file...
#...taking the information we want from it
for i in range(len(players_json)):
    code = players_json[i]['code']
    ID = players_json[i]['id']
    first = remove_accents(players_json[i]['first_name'])
    web = remove_accents(players_json[i]['web_name'])
    team = players_json[i]['team']
    position = players_json[i]['element_type']

    df_temp = pd.DataFrame({'PlayerCode':[code],
                            'PlayerID':[ID],
                            'FirstName':[first],
                            'WebName':[web],
                            'Team':[team],
                            'Position':[position]})

    df_players = pd.concat([df_players, df_temp])

#We set the playerID as the index for the dataframe
df_players.set_index('PlayerID',drop=True,inplace=True)

#We have a list of teams and positions taken from site commentary...
teams = ['Arsenal', 'Aston Villa', 'Bournemouth', 'Brighton and Hove Albion',
         'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Leicester City',
         'Liverpool', 'Manchester City', 'Manchester United', 'Newcastle United',
         'Norwich City', 'Sheffield United', 'Southampton', 'Tottenham Hotspur',
         'Watford', 'West Ham United', 'Wolverhampton Wanderers']

#We also add the commentary names to the teams dataframe
df_teams['CommentName'] = teams


positions = ['GKP','DEF','MID','FWD']

#We then update the columns with these strings as required
df_players['Team'] = df_players['Team'].map(lambda x: teams[x-1])
df_players['Position'] = df_players['Position'].map(lambda x: positions[x-1])
df_players['CommentName'] = df_players['FirstName'].map(str) + ' ' + df_players['WebName'].map(str)


#We need to amend some full names in cases where player goes by single name
#e.g. 'Sokratis'
def single_name_update(name):
    #Check if any player's name is just the same name twice
    split = name.split()
    #For players who have the same name twice e.g. 'Sokratis'
    if split[0] == split[1]:
        return ' '.join(split[1:])
    #For players who have two names twice e.g. 'David Luis'
    if (len(split)==4) & (split[:2] == split[2:]):
        return ' '.join(split[2:])

    return name

df_players['CommentName'] = df_players['CommentName'].map(lambda x: single_name_update(x))

In [15]:
#Create the database 'fpl.db' (fantasy premier league!)
import sqlite3
conn = sqlite3.connect('fpl_akshay.db')
#Instantiate a cursor
c = conn.cursor()

In [16]:
create_sql_table(df_teams, c, table_name = 'TeamsBasic', verbose=True)
conn.commit()

CREATE TABLE TeamsBasic (TeamsBasicID INTEGER PRIMARY KEY, TeamID int64, Team TEXT, ShortName TEXT, Strength TEXT, CommentName TEXT, unique (Team, ShortName, Strength, CommentName));

New table added


In [17]:
populate_sql_from_dataframe(df_teams, 'TeamsBasic', c, verbose=True)
conn.commit()

INSERT INTO TeamsBasic (TeamID, Team, ShortName, Strength, CommentName)
                       VALUES (1, "Arsenal", "ARS", 4, "Arsenal");
                       
INSERT INTO TeamsBasic (TeamID, Team, ShortName, Strength, CommentName)
                       VALUES (2, "Aston Villa", "AVL", 3, "Aston Villa");
                       
INSERT INTO TeamsBasic (TeamID, Team, ShortName, Strength, CommentName)
                       VALUES (3, "Bournemouth", "BOU", 3, "Bournemouth");
                       
INSERT INTO TeamsBasic (TeamID, Team, ShortName, Strength, CommentName)
                       VALUES (4, "Brentford", "BRE", 3, "Brighton and Hove Albion");
                       
INSERT INTO TeamsBasic (TeamID, Team, ShortName, Strength, CommentName)
                       VALUES (5, "Brighton", "BHA", 3, "Burnley");
                       
INSERT INTO TeamsBasic (TeamID, Team, ShortName, Strength, CommentName)
                       VALUES (6, "Burnley", "BUR", 2, "Chelsea");
          

In [25]:
def PlayerAPI(player):
    '''
    Takes the player's code, and returns the
    API JSON file associated with them.
    '''

    url = f'https://fantasy.premierleague.com/api/element-summary/{player}/'
    return requests.get(url).json()

In [26]:
def PlayerHistory(player):
    '''
    Takes the player's code, and returns the
    API JSON file associated with their game
    history from the current season.
    '''
    json_file = PlayerAPI(player)
    return json_file['history']

In [27]:
def PlayerAPIStats(player):
    '''
    Takes a playerID and outputs a dataframe of
    the player's statistics derived from the FPL API
    - value
    - total_points
    - minutes
    - bps
    - clean_sheets
    - selected
    - transfers in (net)
    '''
    #Get the JSON file for the player
    history = PlayerHistory(player)

    #Extract the data from this JSON file
    playerID = [int(player) for i in history]
    gameweeks = [i['round'] for i in history]
    dates = [i['kickoff_time'][:10] for i in history]
    points = [i['total_points'] for i in history]
    value = [i['value']/10 for i in history]
    minutes = [i['minutes'] for i in history]
    bps = [i['bps'] for i in history]
    cs = [i['clean_sheets'] for i in history]
    saves = [i['saves'] for i in history]
    selected = [i['selected'] for i in history]
    transfers = [i['transfers_balance'] for i in history]

    df_temp = pd.DataFrame({'PlayerID':playerID,
                            'GameWeek':gameweeks,
                            'Date':dates,
                            'Points':points,
                            'Price':value,
                            'MinutesPlayed':minutes,
                            'CleanSheet':cs,
                            'Saves':saves,
                            'BPS':bps,
                            'SelectedBy':selected,
                            'NetTransfersIn':transfers})

    return df_temp

def PlayersAPIStats(players, gameweeks=None):
    '''
    Takes a list of playerIDs a list of gameweeks
    and outputs a dataframe of the player's statistics
    derived from the FPL API for those weeks
    - points
    - price
    - bps
    - selected
    - transfers in (net)
    '''
    #If no gameweeks specified, then use all
    #gameweeks in the retured JSON file
    if gameweeks == None:
        gameweeks = list(range(1,PlayerAPIStats(players[0])['GameWeek'].max()+1))

    #Create a blank dataframe
    cols = ['PlayerID', 'GameWeek', 'Date', 'Points', 'Price',
            'BPS', 'SelectedBy', 'NetTransfersIn']
    df_temp = pd.DataFrame(columns=cols)

    #Iterate through the players and add their stats to the dataframe
    for i in players:
        new_rows = PlayerAPIStats(i)
        df_temp = pd.concat([df_temp, new_rows])

    #Remove rows we don't want
    df_temp = df_temp.loc[df_temp['GameWeek'].isin(gameweeks)]

    #Make everything numeric
    df_temp = df_temp.apply(pd.to_numeric, errors='ignore')

    #Reset the index
    df_temp.reset_index(inplace=True, drop=True)

    return df_temp

In [28]:
df_api_stats = PlayersAPIStats(list(df_players.index))
create_sql_table(df_api_stats, c, 'PlayerAPIStats')


CREATE TABLE PlayerAPIStats (PlayerAPIStatsID INTEGER PRIMARY KEY, TableIndex TEXT, PlayerID INTEGER, GameWeek INTEGER, Date TEXT, Points INTEGER, Price REAL, BPS INTEGER, SelectedBy INTEGER, NetTransfersIn INTEGER, MinutesPlayed REAL, CleanSheet REAL, Saves REAL, unique (PlayerID, GameWeek, Date, Points, Price, BPS, SelectedBy, NetTransfersIn, MinutesPlayed, CleanSheet, Saves));

New table added


In [29]:
populate_sql_from_dataframe(df_api_stats, 'PlayerAPIStats', c)
conn.commit()

In [30]:
create_sql_table(df_players, c, 'PlayersBasic')
populate_sql_from_dataframe(df_players, 'PlayersBasic', c, verbose=True)
conn.commit()

CREATE TABLE PlayersBasic (PlayersBasicID INTEGER PRIMARY KEY, PlayerID int64, PlayerCode TEXT, FirstName TEXT, WebName TEXT, Team TEXT, Position TEXT, CommentName TEXT, unique (PlayerCode, FirstName, WebName, Team, Position, CommentName));

New table added
INSERT INTO PlayersBasic (PlayerID, PlayerCode, FirstName, WebName, Team, Position, CommentName)
                       VALUES (1, 232223, "Folarin", "Balogun", "Arsenal", "FWD", "Folarin Balogun");
                       
INSERT INTO PlayersBasic (PlayerID, PlayerCode, FirstName, WebName, Team, Position, CommentName)
                       VALUES (2, 58822, "Cedric", "Cedric", "Arsenal", "DEF", "Cedric");
                       
INSERT INTO PlayersBasic (PlayerID, PlayerCode, FirstName, WebName, Team, Position, CommentName)
                       VALUES (3, 153256, "Mohamed", "M.Elneny", "Arsenal", "MID", "Mohamed M.Elneny");
                       
INSERT INTO PlayersBasic (PlayerID, PlayerCode, FirstName, WebName, Team, Position,